In [1]:
import meep as mp
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple
from scipy.optimize import minimize

#Au material script from complex refractive index data

wvl_data = np.array([250.000000, 260.000000, 270.000000, 280.000000, 290.000000, 300.000000,
                              310.000000, 320.000000, 330.000000, 340.000000, 350.000000, 360.000000,
                              370.000000, 380.000000, 390.000000, 400.000000, 410.000000, 420.000000,
                              430.000000, 440.000000, 450.000000, 460.000000, 470.000000, 480.000000,
                              490.000000, 500.000000, 510.000000, 520.000000, 530.000000, 540.000000,
                              550.000000, 560.000000, 570.000000, 580.000000, 590.000000, 600.000000,
                              610.000000, 620.000000, 630.000000, 640.000000, 650.000000, 660.000000,
                              670.000000, 680.000000, 690.000000, 700.000000, 710.000000, 720.000000,
                              730.000000, 740.000000, 750.000000, 760.000000, 770.000000, 780.000000,
                              790.000000, 800.000000, 810.000000, 820.000000, 830.000000, 840.000000,
                              850.000000, 860.000000, 870.000000, 880.000000, 890.000000, 900.000000,
                              910.000000, 920.000000, 930.000000, 940.000000, 950.000000, 960.000000,
                              970.000000, 980.000000, 990.000000, 1000.000000])

#Refractive index data
n_data = np.array([1.486293, 1.517101, 1.602219, 1.678100, 1.750429, 1.800042, 1.830000, 1.836011,
                   1.813111, 1.781199, 1.751161, 1.726732, 1.706249, 1.687926, 1.670782, 1.658000,
                   1.641467, 1.626593, 1.607286, 1.572004, 1.503407, 1.417996, 1.315496, 1.189871,
                   1.018554, 0.849238, 0.698231, 0.571152, 0.475555, 0.397920, 0.355713, 0.318417,
                   0.295010, 0.278152, 0.261664, 0.245531, 0.229740, 0.214278, 0.199131, 0.184288,
                   0.169737, 0.164060, 0.162125, 0.160792, 0.160087, 0.160906, 0.161863, 0.162930,
                   0.164154, 0.166357, 0.168557, 0.170750, 0.172934, 0.175425, 0.178189, 0.180917,
                   0.183609, 0.186265, 0.189334, 0.193206, 0.197006, 0.200737, 0.204403, 0.208008,
                   0.211773, 0.215750, 0.219658, 0.223500, 0.227279, 0.230997, 0.234659, 0.239028,
                   0.243770, 0.248435, 0.253026, 0.257546])

#Absorption coefficient data (imaginary coefficient of complex refractive index)
k_data = np.array([1.660980, 1.759202, 1.824512, 1.873471, 1.904443, 1.919298, 1.916000, 1.897531,
                   1.870450, 1.852625, 1.847116, 1.855697, 1.883198, 1.916907, 1.939990, 1.956000,
                   1.957488, 1.949527, 1.934151, 1.910625, 1.878436, 1.843614, 1.814717, 1.799925,
                   1.820950, 1.892178, 2.027084, 2.185897, 2.374957, 2.553141, 2.695668, 2.832206,
                   2.899935, 2.930925, 2.961718, 2.992315, 3.022714, 3.052916, 3.082919, 3.112725,
                   3.142334, 3.293285, 3.477580, 3.652616, 3.817434, 3.959739, 4.097111, 4.230027,
                   4.358280, 4.474892, 4.588542, 4.699445, 4.807792, 4.914824, 5.020584, 5.124162,
                   5.225688, 5.325278, 5.419440, 5.505117, 5.589481, 5.672590, 5.754499, 5.835259,
                   5.919898, 6.009588, 6.097960, 6.185070, 6.270970, 6.355709, 6.439333, 6.519685,
                   6.597783, 6.674968, 6.751271, 6.826722])


#Fit the data to Lorentz Function and generate Au material

def lorentzfunc(p: np.ndarray, x: np.ndarray) -> np.ndarray:
    """
    Returns the complex ε profile given a set of Lorentzian parameters p
    (σ_0, ω_0, γ_0, σ_1, ω_1, γ_1, ...) for a set of frequencies x.
    """
    N = len(p) // 3
    y = np.zeros(len(x))
    for n in range(N):
        A_n, x_n, g_n = p[3 * n : 3 * n + 3]
        y = y + A_n / (np.square(x_n) - np.square(x) - 1j * x * g_n)
    return y


def lorentzerr(p: np.ndarray, x: np.ndarray, y: np.ndarray) -> float:
    """
    Returns the error (or residual or loss) as the L2 norm
    of the difference of ε(p,x) and y over a set of frequencies x.
    """
    yp = lorentzfunc(p, x)
    val = np.sum(np.square(abs(y - yp)))
    return val


def lorentzfit(
    p0: np.ndarray,
    x: np.ndarray,
    y: np.ndarray,
    tol: float = 1e-25,
    maxeval: float = 10000,
) -> Tuple[np.ndarray, float]:
    """
    Returns the optimal Lorentzian polarizability parameters and error
    which minimize the error in ε(p0,x) relative to y for an initial
    set of Lorentzian polarizability parameters p0 over a set of
    frequencies x using the scipy.optimize algorithm for a relative
    tolerance tol and a maximum number of iterations maxeval.
    """
    result = minimize(
        lambda p: lorentzerr(p, x, y),
        p0,
        method="L-BFGS-B",
        options={"ftol": tol, "maxiter": maxeval},
    )
    popt = result.x
    minf = result.fun
    return popt, minf


if __name__ == "__main__":
    
    n = n_data[:] + 1j * k_data[:]
    eps_inf = 1.1

    eps = np.square(n) - eps_inf

    wl = wvl_data[:]
    wl_min = 250  # minimum wavelength (units of nm)
    wl_max = 1000  # maximum wavelength (units of nm)
    start_idx = np.where(wl > wl_min)
    idx_start = start_idx[0][0]
    end_idx = np.where(wl < wl_max)
    idx_end = end_idx[0][-1] + 1

    # The fitting function is ε(f) where f is the frequency, rather than ε(λ).
    freqs = 1000 / wl  # units of 1/μm
    freqs_reduced = freqs[idx_start:idx_end]
    wl_reduced = wl[idx_start:idx_end]
    eps_reduced = eps[idx_start:idx_end]

    # Fitting parameter: number of Lorentzian terms to use in the fit
    num_lorentzians = 2

    # Number of times to repeat local optimization with random initial values.
    num_repeat = 30

    ps = np.zeros((num_repeat, 3 * num_lorentzians))
    mins = np.zeros(num_repeat)
    for m in range(num_repeat):
        # Initial values for the Lorentzian polarizability terms. Each term
        # consists of three parameters (σ, ω, γ) and is chosen randomly.
        # Note: for the case of no absorption, γ should be set to zero.
        p_rand = [10 ** (np.random.random()) for _ in range(3 * num_lorentzians)]
        ps[m, :], mins[m] = lorentzfit(p_rand, freqs_reduced, eps_reduced, 1e-25, 50000)
        ps_str = "( " + ", ".join(f"{prm:.4f}" for prm in ps[m, :]) + " )"
        print(f"iteration:, {m:3d}, ps_str, {mins[m]:.6f}")

    # Find the best performing set of parameters.
    idx_opt = np.argmin(mins)
    popt_str = "( " + ", ".join(f"{prm:.4f}" for prm in ps[idx_opt]) + " )"
    print(f"optimal:, {popt_str}, {mins[idx_opt]:.6f}")

    # Define a `Medium` class object using the optimal fitting parameters.
    E_susceptibilities = []

    for n in range(num_lorentzians):
        mymaterial_freq, mymaterial_gamma = ps[idx_opt][3 * n + 1:3 * n + 3]

        if mymaterial_freq == 0:
            mymaterial_sigma = ps[idx_opt][3 * n + 0]
            E_susceptibilities.append(
                mp.DrudeSusceptibility(
                    frequency=1.0, gamma=mymaterial_gamma, sigma=mymaterial_sigma
                )
            )
        else:
            mymaterial_sigma = ps[idx_opt][3 * n + 0] / mymaterial_freq**2
            E_susceptibilities.append(
                mp.LorentzianSusceptibility(
                    frequency=mymaterial_freq,
                    gamma=mymaterial_gamma,
                    sigma=mymaterial_sigma,
                )
            )

    Au = mp.Medium(epsilon=eps_inf, E_susceptibilities=E_susceptibilities)
    

iteration:,   0, ps_str, 148.081582
iteration:,   1, ps_str, 787.690920
iteration:,   2, ps_str, 13043.499609
iteration:,   3, ps_str, 13044.553836
iteration:,   4, ps_str, 13047.165209
iteration:,   5, ps_str, 301.160229
iteration:,   6, ps_str, 13046.432877
iteration:,   7, ps_str, 148.081582
iteration:,   8, ps_str, 13044.815111
iteration:,   9, ps_str, 1282.154918
iteration:,  10, ps_str, 148.081582
iteration:,  11, ps_str, 13051.469910
iteration:,  12, ps_str, 148.081582
iteration:,  13, ps_str, 148.081582
iteration:,  14, ps_str, 515.197800
iteration:,  15, ps_str, 514.117069
iteration:,  16, ps_str, 13043.436354
iteration:,  17, ps_str, 148.081582
iteration:,  18, ps_str, 13045.781363
iteration:,  19, ps_str, 148.081582
iteration:,  20, ps_str, 13045.218465
iteration:,  21, ps_str, 148.081582
iteration:,  22, ps_str, 13043.988006
iteration:,  23, ps_str, 148.081582
iteration:,  24, ps_str, 1282.155879
iteration:,  25, ps_str, 148.081582
iteration:,  26, ps_str, 514.056244
iterat

In [2]:
#Nanorod scattering spectrum
#AR Constant

import meep as mp
import numpy as np
import matplotlib.pyplot as plt



def get_spectrum_nanorod(fraction, wvl_min, wvl_max):

    r = 7.25E-3  # radius of cylinder
    l = 25.40E-3 # length of rod
    
    frac = fraction
    l_c = frac*l # length of cylinder
    
    l_e = 0.5*(l-l_c) # length of half ellipsoid
    
    #default medium is water
    default_material = mp.Medium(index=1.3325)
    
    frq_min = 1 / wvl_max
    frq_max = 1 / wvl_min
    frq_cen = 0.5 * (frq_min + frq_max)
    dfrq = frq_max - frq_min
    nfrq = 100
    
    resolution = 10/r
    
    dpml = 2*r
    pml_layers = [mp.PML(thickness=dpml)]
    
    symmetries = [mp.Mirror(mp.Y), mp.Mirror(mp.Z, phase=-1)]
    
    s = 3*l + dpml
    cell_size = mp.Vector3(s, s, s)
    
    # is_integrated=True necessary for any planewave source extending into PML
    sources = [
        mp.Source(
            mp.GaussianSource(frq_cen, fwidth=dfrq, is_integrated=True),
            center=mp.Vector3(-0.5*s + dpml),
            size=mp.Vector3(0, s, s),
            component=mp.Ez,
        )
    ]
    
    
    sim = mp.Simulation(
        resolution=resolution,
        cell_size=cell_size,
        boundary_layers=pml_layers,
        default_material = default_material,
        sources=sources,
        k_point=mp.Vector3(),
        symmetries=symmetries
    )
    
    
    
    box_x1 = sim.add_flux(
        frq_cen,
        dfrq,
        nfrq,
        mp.FluxRegion(center=mp.Vector3(x=-0.5*l), size=mp.Vector3(0, l, l)),
    )
    
    box_x2 = sim.add_flux(
        frq_cen,
        dfrq,
        nfrq,
        mp.FluxRegion(center=mp.Vector3(x=+0.5*l), size=mp.Vector3(0, l, l)),
    )
    
    box_y1 = sim.add_flux(
        frq_cen,
        dfrq,
        nfrq,
        mp.FluxRegion(center=mp.Vector3(y= -0.5*l), size=mp.Vector3(l, 0, l)),
    )
    
    box_y2 = sim.add_flux(
        frq_cen,
        dfrq,
        nfrq,
        mp.FluxRegion(center=mp.Vector3(y= 0.5*l), size=mp.Vector3(l, 0, l)),
    )
    
    box_z1 = sim.add_flux(
        frq_cen,
        dfrq,
        nfrq,
        mp.FluxRegion(center=mp.Vector3(x=-0.5*l), size=mp.Vector3(l, l, 0)),
    )
    
    box_z2 = sim.add_flux(
        frq_cen,
        dfrq,
        nfrq,
        mp.FluxRegion(center=mp.Vector3(x=+0.5*l), size=mp.Vector3(l, l, 0)),
    )
    
    sim.run(until_after_sources=10)
    
    freqs = mp.get_flux_freqs(box_x1)
    box_x1_data = sim.get_flux_data(box_x1)
    box_x2_data = sim.get_flux_data(box_x2)
    box_y1_data = sim.get_flux_data(box_y1)
    box_y2_data = sim.get_flux_data(box_y2)
    box_z1_data = sim.get_flux_data(box_z1)
    box_z2_data = sim.get_flux_data(box_z2)
    
    box_x1_flux0 = mp.get_fluxes(box_x1)
    
    sim.reset_meep()



    

    ellip_1 = mp.Ellipsoid(material=Au,
                           size = mp.Vector3(2*r, 2*l_e, 2*r),
                           center=mp.Vector3(0, 0.5*l_c, 0))
    
    ellip_2 = mp.Ellipsoid(material=Au,
                           size = mp.Vector3(2*r, 2*l_e, 2*r),
                           center=mp.Vector3(0, -0.5*l_c, 0))
    
    #axis says the orentation of the cylinder
    cylinder = mp.Cylinder(material=Au,
                           radius=r,
                           height=l_c,
                           center=mp.Vector3(),
                           axis=mp.Vector3(0, 1, 0))
    
    geometry = [ellip_1, ellip_2, cylinder]
    
    sim = mp.Simulation(
        resolution=resolution,
        cell_size=cell_size,
        boundary_layers=pml_layers,
        default_material = default_material,
        sources=sources,
        k_point=mp.Vector3(),
        symmetries=symmetries,
        geometry=geometry,
    )
    
    
    
    box_x1 = sim.add_flux(
        frq_cen,
        dfrq,
        nfrq,
        mp.FluxRegion(center=mp.Vector3(x=-0.5*l), size=mp.Vector3(0, l, l)),
    )
    
    box_x2 = sim.add_flux(
        frq_cen,
        dfrq,
        nfrq,
        mp.FluxRegion(center=mp.Vector3(x=+0.5*l), size=mp.Vector3(0, l, l)),
    )
    
    box_y1 = sim.add_flux(
        frq_cen,
        dfrq,
        nfrq,
        mp.FluxRegion(center=mp.Vector3(y= -0.5*l), size=mp.Vector3(l, 0, l)),
    )
    
    box_y2 = sim.add_flux(
        frq_cen,
        dfrq,
        nfrq,
        mp.FluxRegion(center=mp.Vector3(y= 0.5*l), size=mp.Vector3(l, 0, l)),
    )
    
    box_z1 = sim.add_flux(
        frq_cen,
        dfrq,
        nfrq,
        mp.FluxRegion(center=mp.Vector3(x=-0.5*l), size=mp.Vector3(l, l, 0)),
    )
    
    box_z2 = sim.add_flux(
        frq_cen,
        dfrq,
        nfrq,
        mp.FluxRegion(center=mp.Vector3(x=+0.5*l), size=mp.Vector3(l, l, 0)),
    )
    
    
    
    sim.load_minus_flux_data(box_x1, box_x1_data)
    sim.load_minus_flux_data(box_x2, box_x2_data)
    sim.load_minus_flux_data(box_y1, box_y1_data)
    sim.load_minus_flux_data(box_y2, box_y2_data)
    sim.load_minus_flux_data(box_z1, box_z1_data)
    sim.load_minus_flux_data(box_z2, box_z2_data)
    
    sim.run(until_after_sources=100)
    
    #flux power through each flux region.
    box_x1_flux = mp.get_fluxes(box_x1)
    box_x2_flux = mp.get_fluxes(box_x2)
    box_y1_flux = mp.get_fluxes(box_y1)
    box_y2_flux = mp.get_fluxes(box_y2)
    box_z1_flux = mp.get_fluxes(box_z1)
    box_z2_flux = mp.get_fluxes(box_z2)
    
    
    #Just want to plot the power of the scattered flux against wavelength
    scatt_flux = (
        np.asarray(box_x1_flux)
        - np.asarray(box_x2_flux)
        + np.asarray(box_y1_flux)
        - np.asarray(box_y2_flux)
        + np.asarray(box_z1_flux)
        - np.asarray(box_z2_flux)
    )

    perc = str(frac*100)
    
    return scatt_flux, freqs, perc
    #return the scattered flux power data and the range of frequencies



In [ ]:
wvl_min = 400E-3
wvl_max = 1000E-3

freq_data = []  #Going to append all raw sim data arrays, to process and plot after the runs
scatt_data = []
percentages = []

fractions = np.linspace(1,0,20)

index = 1

for fraction in fractions:
    print(f"##################### Run {index} Starting #######################")
    scatt_flux, freqs, perc = get_spectrum_nanorod(fraction, wvl_min, wvl_max)
    scatt_data.append(scatt_flux)     #this appends the array as a row.
    freq_data.append(freqs)
    percentages = np.append(percentages, perc)

    index+=1


##################### Run 1 Starting #######################
-----------
Initializing structure...
Padding y to even number of grid points.
Padding z to even number of grid points.
Halving computational cell along direction y
Halving computational cell along direction z
time for choose_chunkdivision = 0.00427318 s
Working in 3D dimensions.
Computational cell is 0.090625 x 0.090625 x 0.090625 with resolution 1379.31
time for set_epsilon = 1.05983 s
-----------
Meep progress: 0.05292500000000001/16.666666507720947 = 0.3% done in 4.0s, 1256.7s to go
on time step 146 (time=0.052925), 0.027421 s/step
Meep progress: 0.13666250000000002/16.666666507720947 = 0.8% done in 8.0s, 969.9s to go
on time step 377 (time=0.136663), 0.0173807 s/step
Meep progress: 0.2204/16.666666507720947 = 1.3% done in 12.0s, 897.1s to go
on time step 608 (time=0.2204), 0.0173305 s/step
Meep progress: 0.3026875/16.666666507720947 = 1.8% done in 16.0s, 866.9s to go
on time step 835 (time=0.302687), 0.01767 s/step
Meep 

run 0 finished at t = 16.667025000000002 (45978 timesteps)
-----------
Initializing structure...
Padding y to even number of grid points.
Padding z to even number of grid points.
Halving computational cell along direction y
Halving computational cell along direction z
time for choose_chunkdivision = 0.00317717 s
Working in 3D dimensions.
Computational cell is 0.090625 x 0.090625 x 0.090625 with resolution 1379.31
     ellipsoid, center = (0,0.0127,0)
          size (0.0145,0,0.0145)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2,2,2)
     ellipsoid, center = (0,-0.0127,0)
          size (0.0145,0,0.0145)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2,2,2)
     cylinder, center = (0,0,0)
          radius 0.00725, height 0.0254, axis (0, 1, 0)
          dielectric constant epsilon diagonal = (2,2,2)
time for set_epsilon = 1.16866 s
time for set_conductivity = 0.018672 s
time for set_conductivity =

run 0 finished at t = 106.6667125 (294253 timesteps)
##################### Run 2 Starting #######################
-----------
Initializing structure...
Padding y to even number of grid points.
Padding z to even number of grid points.
Halving computational cell along direction y
Halving computational cell along direction z
time for choose_chunkdivision = 0.00381088 s
Working in 3D dimensions.
Computational cell is 0.090625 x 0.090625 x 0.090625 with resolution 1379.31
time for set_epsilon = 0.882684 s
-----------
Meep progress: 0.0656125/16.666666507720947 = 0.4% done in 4.0s, 1012.5s to go
on time step 181 (time=0.0656125), 0.0221111 s/step
Meep progress: 0.167475/16.666666507720947 = 1.0% done in 8.0s, 788.3s to go
on time step 463 (time=0.167838), 0.0142295 s/step
Meep progress: 0.26680000000000004/16.666666507720947 = 1.6% done in 12.0s, 738.0s to go
on time step 737 (time=0.267163), 0.0146129 s/step
Meep progress: 0.364675/16.666666507720947 = 2.2% done in 16.0s, 715.6s to go
on ti

run 0 finished at t = 16.667025000000002 (45978 timesteps)
-----------
Initializing structure...
Padding y to even number of grid points.
Padding z to even number of grid points.
Halving computational cell along direction y
Halving computational cell along direction z
time for choose_chunkdivision = 0.00613999 s
Working in 3D dimensions.
Computational cell is 0.090625 x 0.090625 x 0.090625 with resolution 1379.31
     ellipsoid, center = (0,0.011938,0)
          size (0.0145,0.001524,0.0145)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2,2,2)
     ellipsoid, center = (0,-0.011938,0)
          size (0.0145,0.001524,0.0145)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2,2,2)
     cylinder, center = (0,0,0)
          radius 0.00725, height 0.023876, axis (0, 1, 0)
          dielectric constant epsilon diagonal = (2,2,2)
time for set_epsilon = 1.02324 s
time for set_conductivity = 0.0180299 s
time f

run 0 finished at t = 106.6667125 (294253 timesteps)
##################### Run 3 Starting #######################
-----------
Initializing structure...
Padding y to even number of grid points.
Padding z to even number of grid points.
Halving computational cell along direction y
Halving computational cell along direction z
time for choose_chunkdivision = 0.00301218 s
Working in 3D dimensions.
Computational cell is 0.090625 x 0.090625 x 0.090625 with resolution 1379.31
time for set_epsilon = 0.810874 s
-----------
Meep progress: 0.07540000000000001/16.666666507720947 = 0.5% done in 4.0s, 881.9s to go
on time step 208 (time=0.0754), 0.0192685 s/step
Meep progress: 0.18306250000000002/16.666666507720947 = 1.1% done in 8.0s, 721.5s to go
on time step 505 (time=0.183063), 0.0134818 s/step
Meep progress: 0.29000000000000004/16.666666507720947 = 1.7% done in 12.0s, 678.4s to go
on time step 800 (time=0.29), 0.0135617 s/step
Meep progress: 0.3962125/16.666666507720947 = 2.4% done in 16.0s, 657.

...

run 0 finished at t = 16.667025000000002 (45978 timesteps)
-----------
Initializing structure...
Padding y to even number of grid points.
Padding z to even number of grid points.
Halving computational cell along direction y
Halving computational cell along direction z
time for choose_chunkdivision = 0.00378799 s
Working in 3D dimensions.
Computational cell is 0.090625 x 0.090625 x 0.090625 with resolution 1379.31
     ellipsoid, center = (0,0.008509,0)
          size (0.0145,0.008382,0.0145)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2,2,2)
     ellipsoid, center = (0,-0.008509,0)
          size (0.0145,0.008382,0.0145)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2,2,2)
     cylinder, center = (0,0,0)
          radius 0.00725, height 0.017018, axis (0, 1, 0)
          dielectric constant epsilon diagonal = (2,2,2)
time for set_epsilon = 1.28245 s
time for set_conductivity = 0.0150561 s
time f

run 0 finished at t = 106.6667125 (294253 timesteps)
##################### Run 7 Starting #######################
-----------
Initializing structure...
Padding y to even number of grid points.
Padding z to even number of grid points.
Halving computational cell along direction y
Halving computational cell along direction z
time for choose_chunkdivision = 0.00229692 s
Working in 3D dimensions.
Computational cell is 0.090625 x 0.090625 x 0.090625 with resolution 1379.31
time for set_epsilon = 0.852852 s
-----------
Meep progress: 0.07830000000000001/16.666666507720947 = 0.5% done in 4.0s, 848.6s to go
on time step 216 (time=0.0783), 0.0185444 s/step
Meep progress: 0.1881375/16.666666507720947 = 1.1% done in 8.0s, 701.6s to go
on time step 519 (time=0.188138), 0.0132142 s/step
Meep progress: 0.2990625/16.666666507720947 = 1.8% done in 12.0s, 657.5s to go
on time step 825 (time=0.299063), 0.0130812 s/step
Meep progress: 0.40998750000000006/16.666666507720947 = 2.5% done in 16.0s, 635.4s to 

run 0 finished at t = 16.667025000000002 (45978 timesteps)
-----------
Initializing structure...
Padding y to even number of grid points.
Padding z to even number of grid points.
Halving computational cell along direction y
Halving computational cell along direction z
time for choose_chunkdivision = 0.00254917 s
Working in 3D dimensions.
Computational cell is 0.090625 x 0.090625 x 0.090625 with resolution 1379.31
     ellipsoid, center = (0,0.007366,0)
          size (0.0145,0.010668,0.0145)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2,2,2)
     ellipsoid, center = (0,-0.007366,0)
          size (0.0145,0.010668,0.0145)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2,2,2)
     cylinder, center = (0,0,0)
          radius 0.00725, height 0.014732, axis (0, 1, 0)
          dielectric constant epsilon diagonal = (2,2,2)
time for set_epsilon = 1.10189 s
time for set_conductivity = 0.0144379 s
time f

run 0 finished at t = 106.6667125 (294253 timesteps)
##################### Run 8 Starting #######################
-----------
Initializing structure...
Padding y to even number of grid points.
Padding z to even number of grid points.
Halving computational cell along direction y
Halving computational cell along direction z
time for choose_chunkdivision = 0.00239897 s
Working in 3D dimensions.
Computational cell is 0.090625 x 0.090625 x 0.090625 with resolution 1379.31
time for set_epsilon = 0.809981 s
-----------
Meep progress: 0.068875/16.666666507720947 = 0.4% done in 4.0s, 967.0s to go
on time step 190 (time=0.068875), 0.0211204 s/step
Meep progress: 0.16638750000000002/16.666666507720947 = 1.0% done in 8.0s, 795.9s to go
on time step 459 (time=0.166388), 0.0149148 s/step
Meep progress: 0.2697/16.666666507720947 = 1.6% done in 12.0s, 731.5s to go
on time step 744 (time=0.2697), 0.0140529 s/step
Meep progress: 0.37700000000000006/16.666666507720947 = 2.3% done in 16.0s, 693.1s to go
o

run 0 finished at t = 16.667025000000002 (45978 timesteps)
-----------
Initializing structure...
Padding y to even number of grid points.
Padding z to even number of grid points.
Halving computational cell along direction y
Halving computational cell along direction z
time for choose_chunkdivision = 0.00174117 s
Working in 3D dimensions.
Computational cell is 0.090625 x 0.090625 x 0.090625 with resolution 1379.31
     ellipsoid, center = (0,0.005969,0)
          size (0.0145,0.013462,0.0145)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2,2,2)
     ellipsoid, center = (0,-0.005969,0)
          size (0.0145,0.013462,0.0145)
          axes (1,0,0), (0,1,0), (0,0,1)
          dielectric constant epsilon diagonal = (2,2,2)
     cylinder, center = (0,0,0)
          radius 0.00725, height 0.011938, axis (0, 1, 0)
          dielectric constant epsilon diagonal = (2,2,2)
time for set_epsilon = 1.37465 s
time for set_conductivity = 0.0160971 s
time f

In [ ]:
max_wvls = [] #to store the wavelength value where the peak occurs
index = 0

scatt_data = np.asarray(scatt_data)
freq_data = np.asarray(freq_data)    #turn both lists into numpy arrays for easy processing.

#for each row, plot the data, find the max wavelength, and use the percentage value for label.

for run in scatt_data:
    
    scatt_flux = run
    freqs = freq_data[index]
    perc = percentages[index]
    
    plt.plot(1E3/freqs, -1*scatt_flux*1E9, "-", label=f"{100-round(float(perc),2)}%")
    plt.grid(True, which="both", ls="-")
    plt.xlabel("Wavelength, nm")
    plt.ylabel("Absorption Power, nW")
    #plt.yscale("symlog", linthresh=5E-9) #linear threshold says the region about 0 to scale linearly

    plt.legend(loc="upper right", title = "Curvature")
    plt.title("Absorption Spectrum of Au Nanorod: Varying Curvature")
    plt.tight_layout()


    #Frequency position of the peak is the index of the max value in scattered flux data, in frequencies.
    # Doing argmin since the scattering peak is the largest negative value.
    max_wvl = 1E3/freqs[np.argmin(scatt_flux)]
    max_wvls.append([float(perc), max_wvl])

    index += 1 #So the percentage value in the percentages array changes for each row.

plt.savefig(fname="Absorption Spectrum of an Au Nanorod: Varying Curvature",
            dpi=150, 
            bbox_inches="tight")
plt.show()

max_wvls = np.asarray(max_wvls)
for row in max_wvls:
    print(row)


#plot curvature vs. peak position

plt.plot(max_wvls[:,1], 100-max_wvls[:,0], "gx-")
plt.grid(True, which="both", ls="-")
plt.xlabel("Peak Wavelength, nm")
plt.ylabel("Curvature (ellipsoid % of total rod length), %")
plt.title("Nanorod Curvature vs. Peak Wavelength")
plt.tight_layout()
plt.savefig(fname="Nanorod Curvature vs Peak Wavelength",
            dpi=150, 
            bbox_inches="tight")
plt.show()


#Plot of redshift vs. curvature

redshift = max_wvls[:,1]-max_wvls[0,1] #peak position minus 100% cylinder position
curvature = 100-max_wvls[:,0]

plt.plot(curvature, -1*redshift, "bx-")
plt.grid(True, which="both", ls="-")
plt.xlabel("Curvature (ellipsoid % of total rod length), %")
plt.ylabel("Blueshift from 0% curvature, nm")
plt.title("Blueshift vs. Nanorod Curvature")
plt.tight_layout()
plt.savefig(fname="Blueshift vs Nanorod Curvature",
            dpi=150, 
            bbox_inches="tight")
plt.show()

plt.plot(curvature, redshift, "rx-")
plt.grid(True, which="both", ls="-")
plt.xlabel("Curvature (ellipsoid % of total rod length), %")
plt.ylabel("Redshift from 0% curvature, nm")
plt.title("Redshift vs. Nanorod Curvature")
plt.tight_layout()
plt.savefig(fname="Redshift vs Nanorod Curvature",
            dpi=150, 
            bbox_inches="tight")
plt.show()
